In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

### **Download Dataset**

In [2]:
import os
from functions import download_and_extract_zip

# URL of the zip file on Google Drive
url = 'https://drive.google.com/file/d/1iePsqUVQjv_fVZQzFy7pntcTUtGmb_wx/view?usp=sharing'

# Check if the 'PlantVillage' folder exists
if not os.path.exists('PlantVillage') or not os.listdir('PlantVillage'):
    download_and_extract_zip(url)

# **Training and evaluating Models**

In [16]:
import importlib
import model_fun
importlib.reload(model_fun); 

In [18]:
from model_fun import load_data, SimpleCNN, train_model, evaluate_model, load_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
num_classes = 38

In [19]:
image_width = 32
train_loader, val_loader, test_loader, class_names = load_data(image_width)

model = SimpleCNN(num_classes, image_width, num_layers=5, first_layer_filters=64).to(device)  # Move model to device

train_model(model, num_classes, train_loader, val_loader, epoch = 50)

Epoch 1/50: 100%|██████████| 1358/1358 [01:36<00:00, 14.12batch/s, loss=0.865]


Epoch 1, Average Loss: 1.5623
Validation Accuracy: 76.54%
Validation Loss: 0.7721


Epoch 2/50: 100%|██████████| 1358/1358 [01:36<00:00, 14.14batch/s, loss=0.218]


Epoch 2, Average Loss: 0.5667
Validation Accuracy: 87.96%
Validation Loss: 0.3880


Epoch 3/50: 100%|██████████| 1358/1358 [01:36<00:00, 14.08batch/s, loss=0.0993]


...

Epoch 10, Average Loss: 0.1194
Validation Accuracy: 92.74%
Validation Loss: 0.2983


Epoch 11/50: 100%|██████████| 1358/1358 [01:36<00:00, 14.10batch/s, loss=0.0637]  


Epoch 11, Average Loss: 0.1151
Validation Accuracy: 94.01%
Validation Loss: 0.2773


Epoch 12/50: 100%|██████████| 1358/1358 [01:37<00:00, 13.99batch/s, loss=0.00194] 


Epoch 12, Average Loss: 0.1084
Validation Accuracy: 94.28%
Validation Loss: 0.2484
Early stopping triggered after 12 epochs.


In [20]:
cm, report = evaluate_model(model, test_loader, class_names)

f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

F1 Score: 0.9475
Accuracy: 0.9478


## **Finding best hyperparameters**

In [21]:
from itertools import product

# Parametri da testare
image_sizes = [32]
layers = [2, 3, 4, 5]
first_layer_filters = [8, 16, 32, 64]

# Combinazioni di parametri
parameter_combinations = list(product(image_sizes, layers, first_layer_filters))

In [ ]:
results = []  # Per salvare i risultati

for image_size, num_layers, first_layer_filters in parameter_combinations:
    
    print(f"\n\n--Testing configuration: image_size={image_size}, layers={num_layers}, first_layer_filters={first_layer_filters}")
    
    train_loader, val_loader, test_loader, class_names = load_data(image_width)

    model = SimpleCNN(num_classes, image_width, num_layers, first_layer_filters).to(device)  # Move model to device

    train_model(model, num_classes, train_loader, val_loader, epoch = 50)

    # Valuta il modello
    cm, report = evaluate_model(model, val_loader, class_names)

    # Salva i risultati
    results.append({
        "image_size": image_size,
        "num_layers": num_layers,
        "first_layer_filters": first_layer_filters,
        "confusion_matrix": cm,
        "classification_report": report
    })



--Testing configuration: image_size=32, layers=2, first_layer_filters=8


Epoch 1/50: 100%|██████████| 1358/1358 [00:45<00:00, 29.76batch/s, loss=0.858]


Epoch 1, Average Loss: 1.4828
Validation Accuracy: 75.53%
Validation Loss: 0.8019


Epoch 2/50:  96%|█████████▌| 1303/1358 [00:44<00:01, 47.07batch/s, loss=0.922]

## **Model trained on downscaled images vs fullscaled**

In [ ]:
model = load_model('plant_disease_model_32_3layer_32initial.pth', num_classes, image_width=32, num_layers=3, first_layer_filters=32)
train_loader, val_loader, test_loader, class_names = load_data(32)

_, report = evaluate_model(model, test_loader, class_names)

f1 = report['weighted avg']['f1-score']

print(f"F1 Score with downscaled images (32x32): {f1:.4f}")

F1 Score with downscaled images (32x32): 0.9602


In [ ]:
model = load_model('?', num_classes, image_width=256, num_layers=3, first_layer_filters=32)
train_loader, val_loader, test_loader, class_names = load_data(256)

_, report = evaluate_model(model, test_loader, class_names)
f1 = report['weighted avg']['f1-score']

print(f"F1 Score with original images: {f1}")

## **Analysis of mismatched classes**

## **Heat Map**

## **Multilabel test with missing classes**

## **Test on real Dataset**